In [299]:
import pandas as pd
import numpy as np
import nltk
import re

In [300]:
business_df_key = pd.read_csv(r'C:/Users/abdul/desktop/business_With_key1.csv')

In [301]:
business_df_key.head()

,Unnamed: 0,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,review_count,stars,clean_name,key
0,0,"1775 E Tropicana Ave, Ste 29","{'OutdoorSeating': 'False', 'BusinessAcceptsCreditCards': 'True', 'RestaurantsDelivery': 'False', 'RestaurantsReservations': 'True', 'RestaurantsAttire': ""'casual'"", 'Ambience': ""{'romantic': True...",PZ-LZzSlhSe9utkQYU8pFg,"Restaurants, Italian",Las Vegas,NaN,0,36.100016,-115.128528,Carluccio's Tivoli Gardens,89119.0,40,4.0,carluccio tivoli gardens,['PR0009962' 'PR0009961']
1,1,6007 Dean Martin Dr,"{'BikeParking': 'False', 'Ambience': ""{'romantic': False, 'intimate': True, 'classy': False, 'hipster': False, 'divey': False, 'touristy': False, 'trendy': False, 'upscale': False, 'casual': False...",KWywu2tTEPWmR9JnBc0WyQ,"Nightlife, Arts & Entertainment, Bars, Strip Clubs, Adult Entertainment, Dance Clubs",Las Vegas,"{'Thursday': '19:30-2:0', 'Friday': '19:30-3:0', 'Saturday': '19:30-3:0', 'Sunday': '19:30-2:0'}",1,36.080168,-115.182756,Hunk Mansion,89118.0,107,4.0,hunk mansion,NaN
2,2,6055 E Lake Mead Blvd,"{'BikeParking': 'True', 'BusinessParking': ""{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}"", 'BusinessAcceptsCreditCards': 'True', 'GoodForKids': 'True', 'Happ...",tstimHoMcYbkSC4eBA1wEg,"Mexican, Restaurants, Patisserie/Cake Shop, Food, Bars, Nightlife",Las Vegas,"{'Monday': '11:0-21:0', 'Tuesday': '10:0-21:0', 'Wednesday': '10:0-21:0', 'Thursday': '10:0-21:0', 'Friday': '10:0-21:0', 'Saturday': '10:0-21:0', 'Sunday': '10:0-20:0'}",1,36.195615,-115.040529,Maria's Mexican Restaurant & Bakery,89156.0,184,4.5,maria mexican,['PR0011933']
3,3,4190 N Pecos Rd,"{'RestaurantsPriceRange2': '1', 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': ""{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}"", 'GoodForKids': 'True'}",NBn4hgfGtNz91k3VsDZlmw,"Local Services, Food Banks, Community Service/Non-Profit",Las Vegas,"{'Monday': '0:0-0:0', 'Tuesday': '8:0-17:0', 'Wednesday': '8:0-17:0', 'Thursday': '8:0-17:0', 'Friday': '8:0-17:0', 'Saturday': '8:0-17:0', 'Sunday': '8:0-17:0'}",1,36.236053,-115.096454,Three Square,89115.0,44,5.0,three square,['PR0105994']
4,4,6125 Spring Mountain Rd,"{'RestaurantsPriceRange2': '1', 'Ambience': ""{'romantic': False, 'intimate': False, 'classy': False, 'hipster': False, 'divey': False, 'touristy': False, 'trendy': False, 'upscale': False, 'casual...",kANF0dbeoW34s2vwh6Umfw,"Fast Food, Food, Restaurants, Ice Cream & Frozen Yogurt",Las Vegas,NaN,0,36.125031,-115.225620,Dairy Queen,89146.0,33,2.0,dairy queen,['PR0017094']


In [302]:
business_df_key.drop(columns=['Unnamed: 0'], axis=1, inplace=True)

In [303]:
business_df_key.isna().sum()

address          205
attributes       299
business_id        0
categories         0
city               0
hours           2514
is_open            0
latitude           0
longitude          0
name               0
postal_code       23
review_count       0
stars              0
clean_name       247
key             4655
dtype: int64

In [304]:
business_df_key.shape

(10682, 15)

In [305]:
#removing rows with no keys (keys are permit_number)
business_key_clean = business_df_key.dropna(subset=['key']).reset_index(drop= True)

In [306]:
#removing brackets in the key string
business_key_clean['key'] = business_key_clean['key'].apply(lambda x: re.sub("[\[''\]]",'', x))

In [307]:
new_df = []
#business_id and permit_number of similar restaurant names added to the list
#business_id can have multiple permit_numbers
#the key is split and each permit_number is added in a new row
for i in business_key_clean.index:
    for permit in business_key_clean.loc[i, 'key'].split(' '):
        new_df.append([business_key_clean.loc[i, 'business_id'], permit])
        
new_df = pd.DataFrame(new_df)

In [308]:
#removing '\n' at the end of some permit_numbers
new_df[1] = new_df[1].apply(lambda x: x if '\n' not in x else x[:-2])

In [309]:
new_df.columns = ['business_id', 'permit_number']

In [310]:
bus_df = pd.read_csv(r'C:/Users/abdul/desktop/Cleaned Business.csv')

In [311]:
#Using 'business_id' to join business.json with mapping file
bus_with_mappings = pd.merge(new_df, bus_df, on = 'business_id', how = 'inner')

In [312]:
est_df = pd.read_csv(r'C:/Users/abdul/desktop/Cleaned Establishement.csv')

In [313]:
est_df.head()

,permit_number,facility_id,PE,restaurant_name,location_name,address,latitude,longitude,city_id,city_name,zip_code,nciaa,record_status,current_grade,current_demerits,date_current,previous_grade,date_previous,search_text,Unnamed: 21
0,PR0000002,FA0002677,1006,McDonald's #3549 D HOTEL,McDonalds (Store #3549) @ D HOTEL,301 Fremont St,36.169390,-115.143084,10,Las Vegas,89101-5600,Y,1,A,6.0,2019-04-02 00:00:00,A,2019-04-02 00:00:00,McDonald's #3549 D HOTEL McDonalds 3549 D HOTEL,NaN
1,PR0000003,FA0002678,1008,MCDONALD'S #19724,McDonalds Corporation,4400 E Charleston Blvd,36.159440,-115.079750,10,Las Vegas,89104-5506,Y,1,A,3.0,2019-01-03 00:00:00,A,2019-01-03 00:00:00,MCDONALD'S #19724 MCDONALDS 19724,NaN
2,PR0000004,FA0002679,1007,McDonald's #6965,McDonalds #6965,3175 N Rancho Dr,36.216746,-115.212421,10,Las Vegas,89130-3302,Y,1,A,0.0,2019-06-28 00:00:00,A,2019-06-28 00:00:00,McDonald's #6965 McDonalds 6965,NaN
3,PR0000005,FA0002680,1007,MCDONALD'S #2922,McDonalds (Store #2922),2020 N Rainbow Blvd,36.196385,-115.240740,10,Las Vegas,89108-7049,Y,1,A,0.0,2019-12-10 00:00:00,A,2019-12-10 00:00:00,MCDONALD'S #2922 MCDONALDS 2922,NaN
4,PR0000006,FA0002681,1006,MCDONALD'S #3153 @ PLAZA,McDonalds (Store #3153) @ PLAZA,1 S Main St,36.172176,-115.146829,10,Las Vegas,89101-6370,Y,1,A,3.0,2019-10-07 00:00:00,A,2019-10-07 00:00:00,MCDONALD'S #3153 @ PLAZA MCDONALDS 3153 PLAZA,NaN


In [314]:
#Using permit_number to join establishments.csv with mapping file
business_est = pd.merge(bus_with_mappings, est_df, on = 'permit_number', how = 'inner')

In [315]:
business_est.drop('Unnamed: 21', axis = 1, inplace = True)

In [316]:
business_est.head()

,business_id,permit_number,name,address_x,city,state,postal_code,latitude_x,longitude_x,stars,review_count,is_open,attributes,categories,hours,facility_id,PE,restaurant_name,location_name,address_y,latitude_y,longitude_y,city_id,city_name,zip_code,nciaa,record_status,current_grade,current_demerits,date_current,previous_grade,date_previous,search_text
0,PZ-LZzSlhSe9utkQYU8pFg,PR0009962,Carluccio's Tivoli Gardens,"1775 E Tropicana Ave, Ste 29",Las Vegas,NV,89119.0,36.100016,-115.128528,4.0,40,0,"{'OutdoorSeating': 'False', 'BusinessAcceptsCreditCards': 'True', 'RestaurantsDelivery': 'False', 'RestaurantsReservations': 'True', 'RestaurantsAttire': ""'casual'"", 'Ambience': ""{'romantic': True...","Restaurants, Italian",NaN,FA0005923,1016,Carluccio's Lounge Bar,Carluccios,1775 E Tropicana Ave,36.100723,-115.128564,10,Las Vegas,89119-6529,Y,2,A,4.0,2011-02-01 00:00:00,P,2010-01-27 00:00:00,Carluccio's Lounge Bar Carluccios Lounge Bar
1,PZ-LZzSlhSe9utkQYU8pFg,PR0009961,Carluccio's Tivoli Gardens,"1775 E Tropicana Ave, Ste 29",Las Vegas,NV,89119.0,36.100016,-115.128528,4.0,40,0,"{'OutdoorSeating': 'False', 'BusinessAcceptsCreditCards': 'True', 'RestaurantsDelivery': 'False', 'RestaurantsReservations': 'True', 'RestaurantsAttire': ""'casual'"", 'Ambience': ""{'romantic': True...","Restaurants, Italian",NaN,FA0005923,1003,Carluccio's Restaurant,Carluccios,1775 E Tropicana Ave,36.100723,-115.128564,10,Las Vegas,89119-6529,Y,2,A,10.0,2011-02-01 00:00:00,P,2010-01-27 00:00:00,Carluccio's Restaurant Carluccios Restaurant
2,tstimHoMcYbkSC4eBA1wEg,PR0011933,Maria's Mexican Restaurant & Bakery,6055 E Lake Mead Blvd,Las Vegas,NV,89156.0,36.195615,-115.040529,4.5,184,1,"{'BikeParking': 'True', 'BusinessParking': ""{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}"", 'BusinessAcceptsCreditCards': 'True', 'GoodForKids': 'True', 'Happ...","Mexican, Restaurants, Patisserie/Cake Shop, Food, Bars, Nightlife","{'Monday': '11:0-21:0', 'Tuesday': '10:0-21:0', 'Wednesday': '10:0-21:0', 'Thursday': '10:0-21:0', 'Friday': '10:0-21:0', 'Saturday': '10:0-21:0', 'Sunday': '10:0-20:0'}",FA0006354,1006,Mama Maria's,Mama Marias,6055 E Lake Mead Blvd C,36.195696,-115.040472,10,Las Vegas,89156-7801,Y,1,A,6.0,2019-04-16 00:00:00,A,2019-04-16 00:00:00,Mama Maria's Mama Marias
3,NBn4hgfGtNz91k3VsDZlmw,PR0105994,Three Square,4190 N Pecos Rd,Las Vegas,NV,89115.0,36.236053,-115.096454,5.0,44,1,"{'RestaurantsPriceRange2': '1', 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': ""{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}"", 'GoodForKids': 'True'}","Local Services, Food Banks, Community Service/Non-Profit","{'Monday': '0:0-0:0', 'Tuesday': '8:0-17:0', 'Wednesday': '8:0-17:0', 'Thursday': '8:0-17:0', 'Friday': '8:0-17:0', 'Saturday': '8:0-17:0', 'Sunday': '8:0-17:0'}",FA0069108,1090,THREE SQUARE CATERING,THREE SQUARE - SFSP,4190 N PECOS Rd,36.236076,-115.096567,10,Las Vegas,89115,NaN,2,A,3.0,2019-01-22 00:00:00,A,2018-01-30 00:00:00,THREE SQUARE CATERING THREE SQUARE CATERING
4,kANF0dbeoW34s2vwh6Umfw,PR0017094,Dairy Queen,6125 Spring Mountain Rd,Las Vegas,NV,89146.0,36.125031,-115.225620,2.0,33,0,"{'RestaurantsPriceRange2': '1', 'Ambience': ""{'romantic': False, 'intimate': False, 'classy': False, 'hipster': False, 'divey': False, 'touristy': False, 'trendy': False, 'upscale': False, 'casual...","Fast Food, Food, Restaurants, Ice Cream & Frozen Yogurt",NaN,FA0008571,1007,Dairy Queen #17798,Dairy Queen #17798,6125 Spring Mountain Rd,36.125031,-115.225620,10,Las Vegas,89146-8802,Y,2,A,10.0,2016-03-10 00:00:00,P,2015-07-30 00:00:00,Dairy Queen #17798 Dairy Queen 17798


In [317]:
business_est.shape

(12936, 33)

In [318]:
# reading restaurant inspections data
Inspections = pd.read_csv(r"C:/Users/abdul/Desktop/restaurant_inspections.csv", delimiter=";")

In [319]:
Inspections.shape

(261892, 16)

In [320]:
Inspections.head()

,serial_number,permit_number,facility_id,owner_id,inspection_date,inspection_time,employee_id,inspection_type_id,inspection_demerits,inspection_grade,inspection_grade_new,permit_status,inspection_result,violations,record_updated,Unnamed: 15
0,DA0001121,PR0000411,FA0002433,NaN,2005-03-25 00:00:00,2005-03-25 10:20:00,EE7000535,916,4.0,A,A,NaN,Compliant,113,2013-02-21 22:26:12,NaN
1,DA0001122,PR0000411,FA0002433,NaN,2005-11-07 00:00:00,2005-11-07 10:40:00,EE7000664,916,40.0,C,C,NaN,C Downgrade,"12,14,21,22,25,26,27,28,30,31,36,37,64,111,112,114",2013-02-21 22:26:12,NaN
2,DA0001123,PR0000411,FA0002433,NaN,2005-11-18 00:00:00,2005-11-18 11:40:00,EE7000352,916,0.0,A,A,NaN,Compliant,NaN,2013-02-21 22:26:12,NaN
3,DA0001124,PR0000411,FA0002433,NaN,2006-09-25 00:00:00,2006-09-25 10:25:00,EE7000735,916,10.0,A,A,NaN,Compliant,"4,64",2013-02-21 22:26:12,NaN
4,DA0001125,PR0000411,FA0002433,NaN,2007-07-25 00:00:00,2007-07-25 13:50:00,EE0000001,916,3.0,A,A,NaN,Compliant,"31,35,37",2013-02-21 22:26:12,NaN


In [321]:
Inspections.drop(columns=["Unnamed: 15"], axis=1, inplace=True)

In [322]:
# Reading inspection-type data
Inspection_types = pd.read_csv(r"C:/Users/abdul/Desktop/restaurant_inspection_types.csv", delimiter=";")

In [323]:
Inspection_types.head()

,inspection_type_id,inspection_type,Unnamed: 2
0,604,Epidemiological Investigation,NaN
1,902,Complaint Investigation,NaN
2,914,Re-inspection,NaN
3,916,Routine Inspection,NaN
4,918,Special Request Inspection,NaN


In [324]:
# cleaning
Inspection_types.drop(Inspection_types.columns[2], axis=1, inplace=True)

In [325]:
def display_dataframe_without_trim():
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    pd.set_option('display.max_colwidth', 200)
    
display_dataframe_without_trim()

In [326]:
# performing a join to get inspection types
Inspections = pd.merge(Inspections, Inspection_types, on="inspection_type_id")

In [327]:
Inspections.columns

Index(['serial_number', 'permit_number', 'facility_id', 'owner_id',
       'inspection_date', 'inspection_time', 'employee_id',
       'inspection_type_id', 'inspection_demerits', 'inspection_grade',
       'inspection_grade_new', 'permit_status', 'inspection_result',
       'violations', 'record_updated', 'inspection_type'],
      dtype='object')

In [328]:
business_est.columns

Index(['business_id', 'permit_number', 'name', 'address_x', 'city', 'state',
       'postal_code', 'latitude_x', 'longitude_x', 'stars', 'review_count',
       'is_open', 'attributes', 'categories', 'hours', 'facility_id', 'PE',
       'restaurant_name', 'location_name', 'address_y', 'latitude_y',
       'longitude_y', 'city_id', 'city_name', 'zip_code', 'nciaa',
       'record_status', 'current_grade', 'current_demerits', 'date_current',
       'previous_grade', 'date_previous', 'search_text'],
      dtype='object')

In [329]:
business_est.head(20)

,business_id,permit_number,name,address_x,city,state,postal_code,latitude_x,longitude_x,stars,review_count,is_open,attributes,categories,hours,facility_id,PE,restaurant_name,location_name,address_y,latitude_y,longitude_y,city_id,city_name,zip_code,nciaa,record_status,current_grade,current_demerits,date_current,previous_grade,date_previous,search_text
0,PZ-LZzSlhSe9utkQYU8pFg,PR0009962,Carluccio's Tivoli Gardens,"1775 E Tropicana Ave, Ste 29",Las Vegas,NV,89119.0,36.100016,-115.128528,4.0,40,0,"{'OutdoorSeating': 'False', 'BusinessAcceptsCreditCards': 'True', 'RestaurantsDelivery': 'False', 'RestaurantsReservations': 'True', 'RestaurantsAttire': ""'casual'"", 'Ambience': ""{'romantic': True...","Restaurants, Italian",NaN,FA0005923,1016,Carluccio's Lounge Bar,Carluccios,1775 E Tropicana Ave,36.100723,-115.128564,10,Las Vegas,89119-6529,Y,2,A,4.0,2011-02-01 00:00:00,P,2010-01-27 00:00:00,Carluccio's Lounge Bar Carluccios Lounge Bar
1,PZ-LZzSlhSe9utkQYU8pFg,PR0009961,Carluccio's Tivoli Gardens,"1775 E Tropicana Ave, Ste 29",Las Vegas,NV,89119.0,36.100016,-115.128528,4.0,40,0,"{'OutdoorSeating': 'False', 'BusinessAcceptsCreditCards': 'True', 'RestaurantsDelivery': 'False', 'RestaurantsReservations': 'True', 'RestaurantsAttire': ""'casual'"", 'Ambience': ""{'romantic': True...","Restaurants, Italian",NaN,FA0005923,1003,Carluccio's Restaurant,Carluccios,1775 E Tropicana Ave,36.100723,-115.128564,10,Las Vegas,89119-6529,Y,2,A,10.0,2011-02-01 00:00:00,P,2010-01-27 00:00:00,Carluccio's Restaurant Carluccios Restaurant
2,tstimHoMcYbkSC4eBA1wEg,PR0011933,Maria's Mexican Restaurant & Bakery,6055 E Lake Mead Blvd,Las Vegas,NV,89156.0,36.195615,-115.040529,4.5,184,1,"{'BikeParking': 'True', 'BusinessParking': ""{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}"", 'BusinessAcceptsCreditCards': 'True', 'GoodForKids': 'True', 'Happ...","Mexican, Restaurants, Patisserie/Cake Shop, Food, Bars, Nightlife","{'Monday': '11:0-21:0', 'Tuesday': '10:0-21:0', 'Wednesday': '10:0-21:0', 'Thursday': '10:0-21:0', 'Friday': '10:0-21:0', 'Saturday': '10:0-21:0', 'Sunday': '10:0-20:0'}",FA0006354,1006,Mama Maria's,Mama Marias,6055 E Lake Mead Blvd C,36.195696,-115.040472,10,Las Vegas,89156-7801,Y,1,A,6.0,2019-04-16 00:00:00,A,2019-04-16 00:00:00,Mama Maria's Mama Marias
3,NBn4hgfGtNz91k3VsDZlmw,PR0105994,Three Square,4190 N Pecos Rd,Las Vegas,NV,89115.0,36.236053,-115.096454,5.0,44,1,"{'RestaurantsPriceRange2': '1', 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': ""{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}"", 'GoodForKids': 'True'}","Local Services, Food Banks, Community Service/Non-Profit","{'Monday': '0:0-0:0', 'Tuesday': '8:0-17:0', 'Wednesday': '8:0-17:0', 'Thursday': '8:0-17:0', 'Friday': '8:0-17:0', 'Saturday': '8:0-17:0', 'Sunday': '8:0-17:0'}",FA0069108,1090,THREE SQUARE CATERING,THREE SQUARE - SFSP,4190 N PECOS Rd,36.236076,-115.096567,10,Las Vegas,89115,NaN,2,A,3.0,2019-01-22 00:00:00,A,2018-01-30 00:00:00,THREE SQUARE CATERING THREE SQUARE CATERING
4,kANF0dbeoW34s2vwh6Umfw,PR0017094,Dairy Queen,6125 Spring Mountain Rd,Las Vegas,NV,89146.0,36.125031,-115.225620,2.0,33,0,"{'RestaurantsPriceRange2': '1', 'Ambience': ""{'romantic': False, 'intimate': False, 'classy': False, 'hipster': False, 'divey': False, 'touristy': False, 'trendy': False, 'upscale': False, 'casual...","Fast Food, Food, Restaurants, Ice Cream & Frozen Yogurt",NaN,FA0008571,1007,Dairy Queen #17798,Dairy Queen #17798,6125 Spring Mountain Rd,36.125031,-115.225620,10,Las Vegas,89146-8802,Y,2,A,10.0,2016-03-10 00:00:00,P,2015-07-30 00:00:00,Dairy Queen #17798 Dairy Queen 17798
5,YK5eFV8uS1bCmp8GeiQ-Qg,PR0022046,All Star Donuts & Kolaches,3650 E Flamingo Rd,Las Vegas,NV,89121.0,36.115813,-115.095457,4.0,6,0,"{'RestaurantsPriceRange2': '1', 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': ""{'garage': False, 'street': False, 'validated': False, 'lot': False, 'valet': False}""}","Food, Donuts",NaN,FA

In [330]:
business_est[business_est.permit_number == 'PR0000411']

,business_id,permit_number,name,address_x,city,state,postal_code,latitude_x,longitude_x,stars,review_count,is_open,attributes,categories,hours,facility_id,PE,restaurant_name,location_name,address_y,latitude_y,longitude_y,city_id,city_name,zip_code,nciaa,record_status,current_grade,current_demerits,date_current,previous_grade,date_previous,search_text


In [331]:
business_est.isna().sum()

business_id            0
permit_number          0
name                   0
address_x              5
city                   0
state                  0
postal_code            7
latitude_x             0
longitude_x            0
stars                  0
review_count           0
is_open                0
attributes           199
categories             0
hours               1941
facility_id            0
PE                     0
restaurant_name        0
location_name          0
address_y              0
latitude_y             0
longitude_y            0
city_id                0
city_name              0
zip_code               0
nciaa               2989
record_status          0
current_grade        414
current_demerits     413
date_current         414
previous_grade       428
date_previous        428
search_text            0
dtype: int64

In [332]:
# city has no NAs so if we have a Nan for city after merging with Inspections we can drop it

In [333]:
Inspections = pd.merge(Inspections, business_est, on="permit_number", how="left")

In [334]:
Inspections.head(10)

,serial_number,permit_number,facility_id_x,owner_id,inspection_date,inspection_time,employee_id,inspection_type_id,inspection_demerits,inspection_grade,inspection_grade_new,permit_status,inspection_result,violations,record_updated,inspection_type,business_id,name,address_x,city,state,postal_code,latitude_x,longitude_x,stars,review_count,is_open,attributes,categories,hours,facility_id_y,PE,restaurant_name,location_name,address_y,latitude_y,longitude_y,city_id,city_name,zip_code,nciaa,record_status,current_grade,current_demerits,date_current,previous_grade,date_previous,search_text
0,DA0001121,PR0000411,FA0002433,NaN,2005-03-25 00:00:00,2005-03-25 10:20:00,EE7000535,916,4.0,A,A,NaN,Compliant,113,2013-02-21 22:26:12,Routine Inspection,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,DA0001122,PR0000411,FA0002433,NaN,2005-11-07 00:00:00,2005-11-07 10:40:00,EE7000664,916,40.0,C,C,NaN,C Downgrade,"12,14,21,22,25,26,27,28,30,31,36,37,64,111,112,114",2013-02-21 22:26:12,Routine Inspection,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,DA0001123,PR0000411,FA0002433,NaN,2005-11-18 00:00:00,2005-11-18 11:40:00,EE7000352,916,0.0,A,A,NaN,Compliant,NaN,2013-02-21 22:26:12,Routine Inspection,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,DA0001124,PR0000411,FA0002433,NaN,2006-09-25 00:00:00,2006-09-25 10:25:00,EE7000735,916,10.0,A,A,NaN,Compliant,"4,64",2013-02-21 22:26:12,Routine Inspection,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,DA0001125,PR0000411,FA0002433,NaN,2007-07-25 00:00:00,2007-07-25 13:50:00,EE0000001,916,3.0,A,A,NaN,Compliant,"31,35,37",2013-02-21 22:26:12,Routine Inspection,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,DA0001128,PR0000411,FA0002433,NaN,2008-01-14 00:00:00,2008-01-14 11:15:00,EE7000832,916,10.0,A,A,NaN,Compliant,"34,37,64",2013-02-21 22:26:12,Routine Inspection,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,DA0001129,PR0000411,FA0002433,NaN,2008-10-07 00:00:00,2008-10-07 12:10:00,EE7000832,916,5.0,A,A,NaN,Compliant,"13,19,31",2013-02-21 22:26:12,Routine Inspection,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,DA0001130,PR0000411,FA0002433,NaN,2009-08-25 00:00:00,2009-08-25 11:25:00,EE7000832,916,6.0,A,A,NaN,Compliant,"35,36,112",2013-02-21 22:26:12,Routine Inspection,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,DA0001131,PR0000411,FA0002433,NaN,2010-03-30 00:00:00,2010-03-30 11:30:00,EE7000392,916,18.0,B,B,NaN,B Downgrade,"4,9,12,27,31,112",2013-02-21 22:26:12,Routine Inspection,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,DA0001132,PR0000411,FA0002433,NaN,2010-04-13 00:00:00,2010-04-13 11:15:00,EE7000392,916,1.0,A,A,NaN,Compliant,37,2013-02-21 22:26:12,Routine Inspection,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [335]:
Inspections = Inspections[Inspections.city.notna()]
Inspections.shape

(155676, 48)

In [336]:
Inspections = Inspections.reset_index(drop=True)

In [337]:
Inspections.columns

Index(['serial_number', 'permit_number', 'facility_id_x', 'owner_id',
       'inspection_date', 'inspection_time', 'employee_id',
       'inspection_type_id', 'inspection_demerits', 'inspection_grade',
       'inspection_grade_new', 'permit_status', 'inspection_result',
       'violations', 'record_updated', 'inspection_type', 'business_id',
       'name', 'address_x', 'city', 'state', 'postal_code', 'latitude_x',
       'longitude_x', 'stars', 'review_count', 'is_open', 'attributes',
       'categories', 'hours', 'facility_id_y', 'PE', 'restaurant_name',
       'location_name', 'address_y', 'latitude_y', 'longitude_y', 'city_id',
       'city_name', 'zip_code', 'nciaa', 'record_status', 'current_grade',
       'current_demerits', 'date_current', 'previous_grade', 'date_previous',
       'search_text'],
      dtype='object')

In [338]:
# we have lots of redundant columns
Inspections.drop(columns=['facility_id_y', 'owner_id', 'permit_status', 'hours', 'restaurant_name', 
                         'location_name', 'address_y','state', 'inspection_type_id', 'latitude_y', 'longitude_y', 'city_id', 'city_name', 
                         'search_text'], inplace=True)

In [339]:
Inspections.head(30)

,serial_number,permit_number,facility_id_x,inspection_date,inspection_time,employee_id,inspection_demerits,inspection_grade,inspection_grade_new,inspection_result,violations,record_updated,inspection_type,business_id,name,address_x,city,postal_code,latitude_x,longitude_x,stars,review_count,is_open,attributes,categories,PE,zip_code,nciaa,record_status,current_grade,current_demerits,date_current,previous_grade,date_previous
0,DA0001712,PR0000426,FA0002435,2007-04-09 00:00:00,2007-04-09 15:25:00,EE7000360,2.0,A,A,Compliant,"31,33",2013-02-21 22:26:12,Routine Inspection,ybq76JnmovyueBmrxef-aQ,Augustus Cafe,"Caesar's Palace, 3570 Las Vegas Blvd",Las Vegas,89136.0,36.116219,-115.174572,3.0,64.0,0.0,"{'RestaurantsGoodForGroups': 'True', 'BusinessParking': ""{'garage': True, 'street': False, 'validated': False, 'lot': False, 'valet': False}"", 'BusinessAcceptsCreditCards': 'True', 'RestaurantsPri...","Restaurants, American (Traditional)",1049.0,89109-8924,Y,1.0,A,0.0,2019-11-08 00:00:00,A,2019-11-08 00:00:00
1,DA0001713,PR0000426,FA0002435,2008-05-29 00:00:00,2008-05-29 14:15:00,EE7000417,0.0,A,A,Compliant,NaN,2013-02-21 22:26:12,Routine Inspection,ybq76JnmovyueBmrxef-aQ,Augustus Cafe,"Caesar's Palace, 3570 Las Vegas Blvd",Las Vegas,89136.0,36.116219,-115.174572,3.0,64.0,0.0,"{'RestaurantsGoodForGroups': 'True', 'BusinessParking': ""{'garage': True, 'street': False, 'validated': False, 'lot': False, 'valet': False}"", 'BusinessAcceptsCreditCards': 'True', 'RestaurantsPri...","Restaurants, American (Traditional)",1049.0,89109-8924,Y,1.0,A,0.0,2019-11-08 00:00:00,A,2019-11-08 00:00:00
2,DA0001714,PR0000426,FA0002435,2009-06-29 00:00:00,2009-06-29 14:20:00,EE7000417,9.0,A,A,Compliant,"4,31,34,37",2013-02-21 22:26:12,Routine Inspection,ybq76JnmovyueBmrxef-aQ,Augustus Cafe,"Caesar's Palace, 3570 Las Vegas Blvd",Las Vegas,89136.0,36.116219,-115.174572,3.0,64.0,0.0,"{'RestaurantsGoodForGroups': 'True', 'BusinessParking': ""{'garage': True, 'street': False, 'validated': False, 'lot': False, 'valet': False}"", 'BusinessAcceptsCreditCards': 'True', 'RestaurantsPri...","Restaurants, American (Traditional)",1049.0,89109-8924,Y,1.0,A,0.0,2019-11-08 00:00:00,A,2019-11-08 00:00:00
3,DA0001715,PR0000426,FA0002435,2010-04-22 00:00:00,2010-04-22 14:10:00,EE7000417,0.0,A,A,Compliant,NaN,2013-02-21 22:26:12,Routine Inspection,ybq76JnmovyueBmrxef-aQ,Augustus Cafe,"Caesar's Palace, 3570 Las Vegas Blvd",Las Vegas,89136.0,36.116219,-115.174572,3.0,64.0,0.0,"{'RestaurantsGoodForGroups': 'True', 'BusinessParking': ""{'garage': True, 'street': False, 'validated': False, 'lot': False, 'valet': False}"", 'BusinessAcceptsCreditCards': 'True', 'RestaurantsPri...","Restaurants, American (Traditional)",1049.0,89109-8924,Y,1.0,A,0.0,2019-11-08 00:00:00,A,2019-11-08 00:00:00
4,DA0001716,PR0000426,FA0002435,2011-03-15 00:00:00,2011-03-15 14:15:00,EE7000567,2.0,A,A,Compliant,NaN,2013-02-21 22:26:12,Routine Inspection,ybq76JnmovyueBmrxef-aQ,Augustus Cafe,"Caesar's Palace, 3570 Las Vegas Blvd",Las Vegas,89136.0,36.116219,-115.174572,3.0,64.0,0.0,"{'RestaurantsGoodForGroups': 'True', 'BusinessParking': ""{'garage': True, 'street': False, 'validated': False, 'lot': False, 'valet': False}"", 'BusinessAcceptsCreditCards': 'True', 'RestaurantsPri...","Restaurants, American (Traditional)",1049.0,89109-8924,Y,1.0,A,0.0,2019-11-08 00:00:00,A,2019-11-08 00:00:00
5,DA0001807,PR0000441,FA0002435,2007-12-05 00:00:00,2007-12-05 15:40:00,EE7000360,0.0,A,A,Compliant,NaN,2013-02-21 22:26:12,Routine Inspection,CdxWhFBtjl0LBqlQEfxX8A,Pronto by Giada,3570 S Las Vegas Blvd,Las Vegas,89109.0,36.116156,-115.175058,4.0,273.0,1.0,"{'GoodForKids': 'True', 'OutdoorSeating': 'False', 'Alcohol': ""u'full_bar'"", 'RestaurantsTakeOut': 'True', 'RestaurantsAttire': ""'casual'"", 'RestaurantsPriceRange2': '2', 'RestaurantsTableService'...","Coffee & Tea, Restaurants, Bakeries, Salad, Sandwiches, Food",1016.0,89109-8924,Y,1.0,A,0.0,2020-01-30 00:00:00,A,2020-01-30 00:00:00
6,DA0001808,PR0000441,FA0002435,

In [340]:
Inspections.isna().sum()

serial_number               0
permit_number               0
facility_id_x               0
inspection_date             0
inspection_time            64
employee_id                 0
inspection_demerits       259
inspection_grade          227
inspection_grade_new      227
inspection_result           0
violations              25062
record_updated              0
inspection_type             0
business_id                 0
name                        0
address_x                  33
city                        0
postal_code                51
latitude_x                  0
longitude_x                 0
stars                       0
review_count                0
is_open                     0
attributes               1598
categories                  0
PE                          0
zip_code                    0
nciaa                   11729
record_status               0
current_grade             903
current_demerits          903
date_current              903
previous_grade            916
date_previ

In [341]:
Inspections[Inspections.inspection_demerits.isna()]

,serial_number,permit_number,facility_id_x,inspection_date,inspection_time,employee_id,inspection_demerits,inspection_grade,inspection_grade_new,inspection_result,violations,record_updated,inspection_type,business_id,name,address_x,city,postal_code,latitude_x,longitude_x,stars,review_count,is_open,attributes,categories,PE,zip_code,nciaa,record_status,current_grade,current_demerits,date_current,previous_grade,date_previous
3309,DA0057298,PR0000301,FA0002761,2011-03-17 00:00:00,2011-03-17 14:05:00,EE7000597,NaN,NaN,NaN,Unspecified,NaN,2013-02-21 22:26:12,Routine Inspection,MNaPOkqm2vKeNG8EqkZWCg,Chicago Joe's,820 S 4th St,Las Vegas,89101.0,36.160979,-115.147845,3.5,415.0,1.0,"{'RestaurantsAttire': ""u'casual'"", 'GoodForKids': 'True', 'RestaurantsTableService': 'True', 'RestaurantsTakeOut': 'True', 'Alcohol': ""u'beer_and_wine'"", 'WiFi': ""u'no'"", 'BusinessAcceptsCreditCar...","Italian, Seafood, Sandwiches, Restaurants",1003.0,89101-6709,Y,1.0,A,10.0,2019-10-09 00:00:00,A,2019-10-09 00:00:00
4921,DA0109398,PR0002464,FA0002217,2010-08-16 00:00:00,2010-08-16 15:40:00,EE7000398,NaN,NaN,NaN,Unspecified,NaN,2013-02-21 22:26:12,Routine Inspection,Q5xyZimM1qeez_yoX7Cf0g,Terrible Herbst,3440 W Russell Rd,Las Vegas,89118.0,36.087188,-115.186415,2.0,16.0,1.0,"{'BusinessParking': ""{'garage': False, 'street': False, 'validated': False, 'lot': False, 'valet': False}"", 'BikeParking': 'True', 'RestaurantsTakeOut': 'False', 'RestaurantsPriceRange2': '2', 'Ca...","Food, Gas Stations, Automotive, Car Wash, Convenience Stores",1006.0,89118-2495,Y,1.0,A,8.0,2019-02-08 00:00:00,A,2019-02-08 00:00:00
5012,DA0111480,PR0002673,FA0002337,2007-10-08 00:00:00,2007-10-08 16:15:00,EE7000538,NaN,NaN,NaN,Unspecified,NaN,2013-02-21 22:26:12,Routine Inspection,WUHziAfNsIe73A43q5Xijg,Chubby's Pub,4702 E Flamingo Rd,Las Vegas,89121.0,36.115744,-115.071450,2.5,5.0,1.0,"{'RestaurantsGoodForGroups': 'True', 'RestaurantsReservations': 'False', 'RestaurantsPriceRange2': '1', 'BusinessAcceptsCreditCards': 'False'}","Nightlife, Bars",1028.0,89121-4709,Y,2.0,A,0.0,2015-03-25 00:00:00,P,2014-07-29 00:00:00
5013,DA0111481,PR0002673,FA0002337,2008-06-27 00:00:00,2008-06-27 14:25:00,EE7000891,NaN,NaN,NaN,Unspecified,NaN,2013-02-21 22:26:12,Routine Inspection,WUHziAfNsIe73A43q5Xijg,Chubby's Pub,4702 E Flamingo Rd,Las Vegas,89121.0,36.115744,-115.071450,2.5,5.0,1.0,"{'RestaurantsGoodForGroups': 'True', 'RestaurantsReservations': 'False', 'RestaurantsPriceRange2': '1', 'BusinessAcceptsCreditCards': 'False'}","Nightlife, Bars",1028.0,89121-4709,Y,2.0,A,0.0,2015-03-25 00:00:00,P,2014-07-29 00:00:00
5014,DA0111482,PR0002673,FA0002337,2009-09-03 00:00:00,2009-09-03 14:45:00,EE7000324,NaN,NaN,NaN,Unspecified,NaN,2013-02-21 22:26:12,Routine Inspection,WUHziAfNsIe73A43q5Xijg,Chubby's Pub,4702 E Flamingo Rd,Las Vegas,89121.0,36.115744,-115.071450,2.5,5.0,1.0,"{'RestaurantsGoodForGroups': 'True', 'RestaurantsReservations': 'False', 'RestaurantsPriceRange2': '1', 'BusinessAcceptsCreditCards': 'False'}","Nightlife, Bars",1028.0,89121-4709,Y,2.0,A,0.0,2015-03-25 00:00:00,P,2014-07-29 00:00:00
5015,DA0111483,PR0002673,FA0002337,2010-08-27 00:00:00,2010-08-27 11:00:00,EE7000924,NaN,NaN,NaN,Unspecified,NaN,2013-02-21 22:26:12,Routine Inspection,WUHziAfNsIe73A43q5Xijg,Chubby's Pub,4702 E Flamingo Rd,Las Vegas,89121.0,36.115744,-115.071450,2.5,5.0,1.0,"{'RestaurantsGoodForGroups': 'True', 'RestaurantsReservations': 'False', 'RestaurantsPriceRange2': '1', 'BusinessAcceptsCreditCards': 'False'}","Nightlife, Bars",1028.0,89121-4709,Y,2.0,A,0.0,2015-03-25 00:00:00,P,2014-07-29 00:00:00
5754,DA0121553,PR0002774,FA0002395,2007-11-30 00:00:00,2007-11-30 16:15:00,EE7000594,NaN,NaN,NaN,Unspecified,NaN,2013-02-21 22:26:12,Routine Inspection,ajoqEHnCZTD8-8GqGLq9-Q,Palms Casino Resort,4321 W Flamingo Rd,Las Vegas,89103.0,36.114483,-115.194814,3.0,1377.0,1.0,"{'RestaurantsReservations': 'True', 'OutdoorSeating': 'True', 'Alcohol': ""u'full_bar'"", 'RestaurantsGoodForGroups': 'True', 'RestaurantsAttire':

In [342]:
# removing erroneous rows
Inspections = Inspections[Inspections.inspection_result != 'Unspecified']

In [343]:
Inspections.shape

(155437, 34)

In [344]:
# how many permit numbers correspond to each facility?
a = Inspections.groupby('facility_id_x')['permit_number'].nunique()
a

facility_id_x
FA0002025      2
FA0002026      1
FA0002028      2
FA0002053      1
FA0002059      1
FA0002060      1
FA0002061      1
FA0002070      1
FA0002071      1
FA0002072      2
FA0002074      1
FA0002076      1
FA0002081      3
FA0002084      1
FA0002085      1
FA0002086      1
FA0002091      1
FA0002092      1
FA0002093      1
FA0002099      3
FA0002100      1
FA0002101      4
FA0002102      1
FA0002103      3
FA0002106      1
FA0002107      1
FA0002108      1
FA0002109      1
FA0002110      1
FA0002111      1
FA0002112      1
FA0002114      1
FA0002115      1
FA0002116      1
FA0002117      1
FA0002118      1
FA0002119      1
FA0002120      1
FA0002121      1
FA0002122      1
FA0002123      1
FA0002124      1
FA0002125      1
FA0002127      1
FA0002128      2
FA0002130      2
FA0002131      2
FA0002132      2
FA0002137      4
FA0002139      7
FA0002146      1
FA0002152      1
FA0002163      1
FA0002169      1
FA0002170      2
FA0002171      2
FA0002172      2
FA0002173      2


In [345]:
# On average how many permit IDs does a facility have
a.mean()

1.8681672025723473

In [346]:
# Max number of permit IDs corresponding to one facility_ID
a[a == a.max()]

facility_id_x
FA0006285    164
Name: permit_number, dtype: int64

In [347]:
# confirming that permit number is unique to only one facility_id
Inspections.groupby('permit_number').facility_id_x.nunique()

permit_number
PR0000002    1
PR0000003    1
PR0000004    1
PR0000005    1
PR0000006    1
PR0000007    1
PR0000011    1
PR0000012    1
PR0000013    1
PR0000014    1
PR0000016    1
PR0000021    1
PR0000022    1
PR0000024    1
PR0000025    1
PR0000034    1
PR0000035    1
PR0000036    1
PR0000047    1
PR0000050    1
PR0000051    1
PR0000056    1
PR0000059    1
PR0000060    1
PR0000061    1
PR0000091    1
PR0000099    1
PR0000113    1
PR0000114    1
PR0000115    1
PR0000116    1
PR0000117    1
PR0000118    1
PR0000119    1
PR0000120    1
PR0000121    1
PR0000122    1
PR0000123    1
PR0000124    1
PR0000127    1
PR0000128    1
PR0000130    1
PR0000131    1
PR0000132    1
PR0000133    1
PR0000134    1
PR0000135    1
PR0000136    1
PR0000137    1
PR0000138    1
PR0000139    1
PR0000140    1
PR0000141    1
PR0000142    1
PR0000143    1
PR0000144    1
PR0000145    1
PR0000146    1
PR0000147    1
PR0000148    1
PR0000160    1
PR0000161    1
PR0000162    1
PR0000244    1
PR0000245    1
PR0000256  

In [348]:
# Out of curiousity
Inspections[Inspections.facility_id_x == 'FA0006285'].head(20)

,serial_number,permit_number,facility_id_x,inspection_date,inspection_time,employee_id,inspection_demerits,inspection_grade,inspection_grade_new,inspection_result,violations,record_updated,inspection_type,business_id,name,address_x,city,postal_code,latitude_x,longitude_x,stars,review_count,is_open,attributes,categories,PE,zip_code,nciaa,record_status,current_grade,current_demerits,date_current,previous_grade,date_previous
4409,DA00ZIVK3,PR0008612,FA0006285,2017-01-31 00:00:00,2017-01-31 18:00:00,EE7001108,5.0,A,A,A Grade,202,2017-02-16 09:26:49,Routine Inspection,JTFFtWy_chbsHTs95yD_2Q,Cafe Bellagio,3600 S Las Vegas Blvd,Las Vegas,89109.0,36.112144,-115.176716,3.5,896.0,0.0,"{'BusinessAcceptsCreditCards': 'True', 'Alcohol': ""'full_bar'"", 'RestaurantsDelivery': 'False', 'GoodForKids': 'True', 'BikeParking': 'False', 'RestaurantsPriceRange2': '2', 'OutdoorSeating': 'Fal...","Sandwiches, Cafes, American (New), Burgers, American (Traditional), Restaurants",1006.0,89109-4303,Y,1.0,A,0.0,2020-01-28 00:00:00,A,2020-01-28 00:00:00
19038,DA0366761,PR0008599,FA0006285,2005-04-22 00:00:00,2005-04-22 14:00:00,EE7000417,1.0,A,A,Compliant,19,2013-02-21 22:26:12,Routine Inspection,JTFFtWy_chbsHTs95yD_2Q,Cafe Bellagio,3600 S Las Vegas Blvd,Las Vegas,89109.0,36.112144,-115.176716,3.5,896.0,0.0,"{'BusinessAcceptsCreditCards': 'True', 'Alcohol': ""'full_bar'"", 'RestaurantsDelivery': 'False', 'GoodForKids': 'True', 'BikeParking': 'False', 'RestaurantsPriceRange2': '2', 'OutdoorSeating': 'Fal...","Sandwiches, Cafes, American (New), Burgers, American (Traditional), Restaurants",1016.0,89109-4303,Y,1.0,A,0.0,2019-05-14 00:00:00,A,2019-05-14 00:00:00
19039,DA0366762,PR0008599,FA0006285,2005-12-22 00:00:00,2005-12-22 14:00:00,EE7000417,0.0,A,A,Compliant,NaN,2013-02-21 22:26:12,Routine Inspection,JTFFtWy_chbsHTs95yD_2Q,Cafe Bellagio,3600 S Las Vegas Blvd,Las Vegas,89109.0,36.112144,-115.176716,3.5,896.0,0.0,"{'BusinessAcceptsCreditCards': 'True', 'Alcohol': ""'full_bar'"", 'RestaurantsDelivery': 'False', 'GoodForKids': 'True', 'BikeParking': 'False', 'RestaurantsPriceRange2': '2', 'OutdoorSeating': 'Fal...","Sandwiches, Cafes, American (New), Burgers, American (Traditional), Restaurants",1016.0,89109-4303,Y,1.0,A,0.0,2019-05-14 00:00:00,A,2019-05-14 00:00:00
19040,DA0366763,PR0008599,FA0006285,2006-05-11 00:00:00,2006-05-11 12:10:00,EE7000417,0.0,A,A,Compliant,NaN,2013-02-21 22:26:12,Routine Inspection,JTFFtWy_chbsHTs95yD_2Q,Cafe Bellagio,3600 S Las Vegas Blvd,Las Vegas,89109.0,36.112144,-115.176716,3.5,896.0,0.0,"{'BusinessAcceptsCreditCards': 'True', 'Alcohol': ""'full_bar'"", 'RestaurantsDelivery': 'False', 'GoodForKids': 'True', 'BikeParking': 'False', 'RestaurantsPriceRange2': '2', 'OutdoorSeating': 'Fal...","Sandwiches, Cafes, American (New), Burgers, American (Traditional), Restaurants",1016.0,89109-4303,Y,1.0,A,0.0,2019-05-14 00:00:00,A,2019-05-14 00:00:00
19041,DA0366764,PR0008599,FA0006285,2007-05-01 00:00:00,2007-05-01 12:20:00,EE7000417,4.0,A,A,Compliant,14,2013-02-21 22:26:12,Routine Inspection,JTFFtWy_chbsHTs95yD_2Q,Cafe Bellagio,3600 S Las Vegas Blvd,Las Vegas,89109.0,36.112144,-115.176716,3.5,896.0,0.0,"{'BusinessAcceptsCreditCards': 'True', 'Alcohol': ""'full_bar'"", 'RestaurantsDelivery': 'False', 'GoodForKids': 'True', 'BikeParking': 'False', 'RestaurantsPriceRange2': '2', 'OutdoorSeating': 'Fal...","Sandwiches, Cafes, American (New), Burgers, American (Traditional), Restaurants",1016.0,89109-4303,Y,1.0,A,0.0,2019-05-14 00:00:00,A,2019-05-14 00:00:00
19042,DA0366765,PR0008599,FA0006285,2008-05-07 00:00:00,2008-05-07 16:20:00,EE7000487,1.0,A,A,Compliant,35,2013-02-21 22:26:12,Routine Inspection,JTFFtWy_chbsHTs95yD_2Q,Cafe Bellagio,3600 S Las Vegas Blvd,Las Vegas,89109.0,36.112144,-115.176716,3.5,896.0,0.0,"{'BusinessAcceptsCreditCards': 'True', 'Alcohol': ""'full_bar'"", 'RestaurantsDelivery': 'False', 'GoodForKids': 'True', 'BikeParking': 'False', 'RestaurantsPriceRange2': '2', 'OutdoorSeating': 'Fal...","Sandwiches, Cafes, Americ

In [349]:
# Mappings file
mappings = pd.read_csv(r"C:/Users/abdul/Desktop/Mapping.csv")

In [350]:
mappings.head()

,business_id,address_x,facility_id,location_name,name,restaurant_name,address_y,is_valid,Unnamed: 8
0,PZ-LZzSlhSe9utkQYU8pFg,"1775 E Tropicana Ave, Ste 29",FA0005923,Carluccios,Carluccio's Tivoli Gardens,Carluccio's Lounge Bar,1775 E Tropicana Ave,y,NaN
1,PZ-LZzSlhSe9utkQYU8pFg,"1775 E Tropicana Ave, Ste 29",FA0005923,Carluccios,Carluccio's Tivoli Gardens,Carluccio's Restaurant,1775 E Tropicana Ave,y,NaN
2,tstimHoMcYbkSC4eBA1wEg,6055 E Lake Mead Blvd,FA0006354,Mama Marias,Maria's Mexican Restaurant & Bakery,Mama Maria's,6055 E Lake Mead Blvd C,y,NaN
3,NBn4hgfGtNz91k3VsDZlmw,4190 N Pecos Rd,FA0069108,THREE SQUARE - SFSP,Three Square,THREE SQUARE CATERING,4190 N PECOS Rd,y,NaN
4,kANF0dbeoW34s2vwh6Umfw,6125 Spring Mountain Rd,FA0008571,Dairy Queen #17798,Dairy Queen,Dairy Queen #17798,6125 Spring Mountain Rd,y,NaN


In [351]:
mappings.is_valid.value_counts()

y     8972
n     3920
N        3
m        2
nn       2
yy       2
t        1
\y       1
yn       1
Name: is_valid, dtype: int64

In [352]:
# cleaning
mappings.is_valid = mappings.is_valid.str.lower()
mappings.is_valid.replace(to_replace=['m', 'nn'], value='n', inplace=True)
mappings.is_valid.replace(to_replace=['yy', '\y'], value='y', inplace=True)
mappings = mappings[(mappings.is_valid  == 'y')| (mappings.is_valid =='n')]

In [353]:
mappings.is_valid.value_counts()

y    8975
n    3927
Name: is_valid, dtype: int64

In [354]:
maps = mappings[mappings.is_valid == 'y']
maps.shape

(8975, 9)

In [355]:
# subsetting Inspections based on mapping
Inspections = Inspections[Inspections.business_id.isin(maps.business_id)]

In [356]:
Inspections.shape

(137875, 34)

In [357]:
# The cases that have NAs for violations are observations where there are 0 demerits
# Lets fill -1 for these cases
Inspections.violations.fillna(-1, inplace=True)

In [358]:
Inspections.isna().sum()

serial_number               0
permit_number               0
facility_id_x               0
inspection_date             0
inspection_time            56
employee_id                 0
inspection_demerits        29
inspection_grade            0
inspection_grade_new        0
inspection_result           0
violations                  0
record_updated              0
inspection_type             0
business_id                 0
name                        0
address_x                   0
city                        0
postal_code                 6
latitude_x                  0
longitude_x                 0
stars                       0
review_count                0
is_open                     0
attributes               1189
categories                  0
PE                          0
zip_code                    0
nciaa                   10688
record_status               0
current_grade             788
current_demerits          788
date_current              788
previous_grade            800
date_previ

In [359]:
Inspections[Inspections.inspection_demerits.isna()]

,serial_number,permit_number,facility_id_x,inspection_date,inspection_time,employee_id,inspection_demerits,inspection_grade,inspection_grade_new,inspection_result,violations,record_updated,inspection_type,business_id,name,address_x,city,postal_code,latitude_x,longitude_x,stars,review_count,is_open,attributes,categories,PE,zip_code,nciaa,record_status,current_grade,current_demerits,date_current,previous_grade,date_previous
56102,DA0840447,PR0017278,FA0009188,2011-07-20 00:00:00,2011-07-20 15:45:00,EE7000366,NaN,A,A,Compliant,-1,2013-02-21 22:26:12,Routine Inspection,sjO0HUb3VdYGQdHxrTnKUw,Del Taco,4380 N Nellis Blvd,Las Vegas,89115.0,36.240000,-115.061403,1.5,4.0,1.0,"{'RestaurantsPriceRange2': '2', 'RestaurantsReservations': 'False', 'BusinessAcceptsCreditCards': 'True', 'RestaurantsTakeOut': 'True', 'GoodForKids': 'True', 'RestaurantsDelivery': 'False', 'Rest...","Mexican, Restaurants",1007.0,89115-2470,Y,1.0,A,9.0,2019-05-23 00:00:00,A,2019-05-23 00:00:00
56103,DA0840447,PR0017278,FA0009188,2011-07-20 00:00:00,2011-07-20 15:45:00,EE7000366,NaN,A,A,Compliant,-1,2013-02-21 22:26:12,Routine Inspection,HXCCITejNvcqtjNQRHnMPw,Del Taco,4380 N Nellis Blvd,Las Vegas,89115.0,36.240009,-115.061402,3.0,7.0,1.0,"{'BikeParking': 'True', 'Alcohol': ""'none'"", 'RestaurantsGoodForGroups': 'True', 'NoiseLevel': ""'average'"", 'HasTV': 'False', 'BusinessAcceptsCreditCards': 'True', 'RestaurantsReservations': 'Fals...","Fast Food, Restaurants, Mexican",1007.0,89115-2470,Y,1.0,A,9.0,2019-05-23 00:00:00,A,2019-05-23 00:00:00
57236,DA0866631,PR0010519,FA0006285,2011-09-09 00:00:00,2011-09-09 12:45:00,EE7000355,NaN,A,A,Compliant,215,2013-02-21 22:26:12,Routine Inspection,JTFFtWy_chbsHTs95yD_2Q,Cafe Bellagio,3600 S Las Vegas Blvd,Las Vegas,89109.0,36.112144,-115.176716,3.5,896.0,0.0,"{'BusinessAcceptsCreditCards': 'True', 'Alcohol': ""'full_bar'"", 'RestaurantsDelivery': 'False', 'GoodForKids': 'True', 'BikeParking': 'False', 'RestaurantsPriceRange2': '2', 'OutdoorSeating': 'Fal...","Sandwiches, Cafes, American (New), Burgers, American (Traditional), Restaurants",1034.0,89109-4303,Y,1.0,A,3.0,2019-08-26 00:00:00,A,2019-08-26 00:00:00
57606,DA0870943,PR0020735,FA0063367,2011-09-16 00:00:00,2011-09-16 16:00:00,EE7000597,NaN,A,A,Compliant,"215,228",2013-02-21 22:26:12,Routine Inspection,pH0BLkL4cbxKzu471VZnuA,SUSHISAMBA - Las Vegas,3327 Las Vegas Blvd S,Las Vegas,89109.0,36.124582,-115.167553,4.0,2355.0,1.0,"{'GoodForDancing': 'False', 'BusinessParking': ""{'garage': True, 'street': False, 'validated': False, 'lot': False, 'valet': False}"", 'GoodForKids': 'False', 'Music': ""{'dj': False, 'background_mu...","Bars, Dim Sum, Japanese, Asian Fusion, Restaurants, Chinese, Breakfast & Brunch, Nightlife",1003.0,89109-8941,Y,1.0,A,3.0,2019-07-22 00:00:00,A,2019-07-22 00:00:00
58094,DA0876922,PR0023365,FA0076552,2011-10-18 00:00:00,2011-10-18 12:50:00,EE7001012,NaN,B,B,B Downgrade,"202,209,212,215,217,230",2015-01-12 09:57:31,Routine Inspection,LPMZ9N1sAjs2nDx7DmiZ2w,Palermo's Pizza,"1370 E Flamingo Rd, Ste F",Las Vegas,89119.0,36.115456,-115.133893,3.0,145.0,1.0,"{'RestaurantsAttire': ""u'casual'"", 'BikeParking': 'True', 'Ambience': ""{'romantic': False, 'intimate': False, 'classy': False, 'hipster': False, 'divey': False, 'touristy': False, 'trendy': False,...","Restaurants, Pizza, Food, Food Delivery Services",1003.0,89119,Y,2.0,A,8.0,2019-01-13 00:00:00,A,2018-08-31 00:00:00
58132,DA0877454,PR0001923,FA0002660,2011-10-20 00:00:00,2011-10-20 13:30:00,EE7000355,NaN,P,PASS,Compliant,-1,2013-02-21 22:26:12,Routine Inspection,lZZLmG1ANLq2Yb3KuVA4NA,Circus Buffet,2880 S Las Vegas Blvd,Las Vegas,89109.0,36.137667,-115.164659,2.0,370.0,1.0,"{'RestaurantsAttire': ""u'casual'"", 'RestaurantsGoodForGroups': 'True', 'RestaurantsTakeOut': 'False', 'OutdoorSeating': 'False', 'RestaurantsPriceRange2': '2', 'GoodForKids': 'True', 'HasTV': 'Fal...","Casinos, Restaurants, Arts & Entertainment, American (New), Buffets",1092.0,89109-1138,Y,2.0,NaN,NaN,NaN,NaN,NaN
58133,DA

In [360]:
# Reading reviews data
reviews = pd.read_csv(r'C:/Users/abdul/Desktop/review.csv')

In [361]:
reviews.shape

(6685900, 9)

In [362]:
# subsetting reviews based on business_id present in Inspections data
reviews = reviews[reviews.business_id.isin(Inspections.business_id)]
reviews.shape

(896214, 9)

In [363]:
reviews.reset_index(drop=True, inplace=True)

In [364]:
reviews.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,IByf6mVY0WA838XNSh-5MA,3CJUJILq7CLHk_9OrvpvQg,I4Nr-MVc26qWr08-S3Q1ow,4.0,0,0,0,"The food is always good and the prices are reasonable. Although it's not exactly a restaurant -- more like a snack bar, limited menu and you walk up to the counter, place your order, wait a minut...",2016-01-17 05:26:22
1,qm97yMwREr7BKkexlwLFbg,TTRVdTXKcq-xn6-1IWbwEw,Gyrez6K8f1AyR7dzW9fvAw,5.0,0,0,1,their pettuccine was fresh-made in the morning. it was very delicate and yummy! the owner and staffs are super kind and delightful. We also had Veal piccata. it was classic! love this restaurant.,2013-12-28 22:28:08
2,0AsmPiAQduxh5jE_si8cLA,C_hUvw2z0R-Rv0yZb6QCZA,dm6sO_Y8JdKTE1ZM955yug,5.0,0,0,0,ended up here because Raku was closed and it received great ratings on Yelp. I'm so glad I came here. One of the better meals I've had. Started off with the mushroom dish and the lettuce wrap. ...,2014-12-17 19:04:33
3,j3vP8537KHvoXNHQIr3haA,Z_HE_KKT7N-WddPTzUQC7A,jScBTQtdAt-8RshaiBEHgw,5.0,1,0,1,"Party of 3 ordered the fish tacos, pork belly banh mi, corn beef hash Benedict, and bam ham. Everything was delicious, esp the pork belly banh mi. It's tender and juicy and the baguette was nicely...",2018-05-28 20:56:05
4,f0B9-r14-bLudyu5S7aLhw,SvMGr_Oih7ivtzmnCL-Tmg,ujHiaprwCQ5ewziu0Vi9rw,1.0,1,0,0,We had dinner at the Bellagio Buffet last night. The service was OK. Our server was great but kind of forgot about us towards the end of our visit. The food was cold. The only good thing there was...,2013-12-07 00:14:06


In [365]:
# Save the 2 datasets as csv
reviews.to_csv(r"C:/Users/abdul/Desktop/reviews_final.csv", encoding='utf-8', index=False)
Inspections.to_csv(r"C:/Users/abdul/Desktop/inspections_final.csv", encoding='utf-8', index=False)